## Performance Metrics Calculation for NWM Data
This script calculates several performance metrics for the NWM streamflow data comparing it to the USGS streamflow data. 
# Input:
- File with list of COMIDs where the metrics are to be calculated
- NWM data (streamflow time  series) for the COMIDs
- USGS data (streamflow time series) for the station on/near the COMIDs. 
- File with the annual maximum streamflow value (for thresholds) at the locations

# Output:
- Excel file with performance metrics

In [ ]:
import os
import pandas as pd
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed
import cProfile
import pstats

# Function to read NWM and USGS data
def read_data(comid, gauge_name, year, start_date, end_date):
    nwm_discharge_path = f'{year}/discharge_{comid}_0{gauge_name}.csv'
    usgs_data_path = f'usgs_data_1979_2023_parquet/discharge_0{gauge_name}.parquet'
    
    # Read NWM discharge data
    nwm_discharge = pd.read_csv(nwm_discharge_path)
    nwm_discharge['Time'] = nwm_discharge['time']
    nwm_discharge = nwm_discharge[(nwm_discharge['Time'] >= start_date) & (nwm_discharge['Time'] <= end_date)]
    nwm_discharge['Discharge_NWM'] = nwm_discharge['streamflow']

    # Read USGS data
    usgs_data = pd.read_parquet(usgs_data_path)
    
    return nwm_discharge, usgs_data


# Function to filter USGS data based on the threshold value
def filter_and_save_data(year, usgs_data, comid, gauge_name, threshold, start_date, end_date):
    usgs_data = usgs_data.copy()

    if usgs_data.empty:
        return pd.DataFrame()

    usgs_data['Time'] = pd.to_datetime(usgs_data['Time'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
    # IF USGS data is downloaded from the NWIS site no time conversion is required. Here we are using USGS data downloaded from website which is in CDT, so conversion is required
    usgs_data['Time'] = usgs_data['Time'].dt.tz_localize('US/Central', ambiguous='NaT')
    usgs_data['Time'] = usgs_data['Time'].dt.tz_convert('UTC')
    usgs_data = usgs_data[usgs_data['Time'].dt.minute == 0]
    usgs_data = usgs_data[(usgs_data['Time'] >= start_date) & (usgs_data['Time'] <= end_date)]
    usgs_data['Discharge'] = pd.to_numeric(usgs_data['Discharge'], errors='coerce') / 35.3147  # Convert from CFS to CMS
    usgs_filtered = usgs_data[usgs_data['Discharge'] >= threshold]

    return usgs_filtered


# Function to calculate Kling-Gupta Efficiency (KGE)
def calculate_kge(observed, modeled):
    mean_observed = np.mean(observed)
    std_observed = np.std(observed)
    mean_modeled = np.mean(modeled)
    std_modeled = np.std(modeled)
    if std_observed == 0 or std_modeled == 0:
            return np.nan
    r = np.corrcoef(observed, modeled)[0, 1]
    alpha = np.std(modeled) / np.std(observed)
    beta = np.mean(modeled) / np.mean(observed)
    kge = 1 - np.sqrt((r - 1) ** 2 + (alpha - 1) ** 2 + (beta - 1) ** 2)
    return kge

# Function to calculate Nash-Sutcliffe Efficiency (NSE)
def calculate_nse(observed, simulated):
    mean_observed = np.mean(observed)
    denominator = np.sum((observed - mean_observed)**2)
    
    if denominator == 0:
        return np.nan  
    
    try:
        nse = 1 - (np.sum((observed - simulated)**2) / denominator)
    except:
        return np.nan
    return nse

# Function to calculate Normalized Nash-Sutcliffe Efficiency (NNSE)
def calculate_nnse(nse, observed):
    mean_observed = np.mean(observed)
    nnse = 1/(2-nse)
    return nnse


# Function to process each row
def process_row(row, year, start_date, end_date, read_data, filter_and_save_data):
    comid, gauge_name, threshold = row['COMID'], row['station_no'], row['Q_cms']
    try:
        # Read NWM and USGS data
        nwm_discharge, usgs_data = read_data(comid, gauge_name, year, start_date, end_date)
        usgs_filtered = filter_and_save_data(year, usgs_data, comid, gauge_name, threshold, start_date, end_date)

        if usgs_filtered.empty:
            return None

        # Convert the time to UTC and remove the timezone information (convert to timezone-naive datetime)
        nwm_discharge['Time'] = pd.to_datetime(nwm_discharge['Time'], utc=True).dt.tz_localize(None).astype('datetime64[ns]')
        usgs_filtered['Time'] = pd.to_datetime(usgs_filtered['Time'], utc=True).dt.tz_localize(None).astype('datetime64[ns]')

        # Using merge_asof to match nearest timestamps between NWM and USGS data
        merged_df = pd.merge_asof(
            usgs_filtered.sort_values('Time'),  # USGS data (sorted by time)
            nwm_discharge.sort_values('Time'),  # NWM data (sorted by time)
            on='Time',  # The time column to merge on
            direction='nearest',  # Match the nearest timestamp
            tolerance=pd.Timedelta('15min')  # Set a tolerance (considered 15 minutes here)
        )

        
        merged_df = merged_df.dropna()

        # Creating the final dataframe for comparison
        new_df_final = pd.DataFrame({'Date': merged_df['Time']})
        new_df_final['Observed'] = merged_df['Discharge']  # USGS discharge

        new_df_final['NWM'] = merged_df['Discharge_NWM']  # NWM discharge
        new_df_final['Observed - Model'] = new_df_final['Observed'] - new_df_final['NWM']

        # Save final data to a CSV file
        output_folder_path = f'result_{year}'
        if not os.path.exists(output_folder_path):
            os.makedirs(output_folder_path)
        new_df_final.to_csv(f'{output_folder_path}/{gauge_name}.csv')

        # Calculate performance metrics
        observed = new_df_final['Observed']
        modeled = new_df_final['NWM']

        # RMSE
        rmse = np.sqrt(np.mean((modeled - observed) ** 2))

        # KGE
        kge = calculate_kge(observed, modeled)

        # PBIAS (Percent Bias)
        pbias = 100 * (np.sum(modeled - observed) / np.sum(observed)) if np.sum(observed) != 0 else np.nan

        # Normalized Error
        # normalized_error = rmse / np.mean(observed) if np.mean(observed) != 0 else np.nan 

        nse = calculate_nse(observed, modeled)
        # Calculate NNSE
        nnse = calculate_nnse(nse, observed)

        # Save results to the original row
        result = row.copy()
        result['RMSE'] = rmse
        result['KGE'] = kge
        result['PBIAS'] = pbias
        result['Discharge'] = np.mean(observed)
        # result['Normalized Error'] = normalized_error
        result['NSE'] = nse
        result['NNSE'] = nnse

        return result

    except Exception as e:
        print(f"An error occurred for COMID {comid}, Gauge {gauge_name}: {e}")
        return None


# Main processing function with parallel execution using ThreadPoolExecutor
def main_parallel_processing(comid_stn, year, start_date, end_date):
    results = []
    
    # Use ThreadPoolExecutor
    with ThreadPoolExecutor(max_workers=4) as executor:
        future_to_row = {executor.submit(process_row, row, year, start_date, end_date, read_data, filter_and_save_data): row for idx, row in comid_stn.iterrows()}
        
        for future in as_completed(future_to_row):
            result = future.result()
            if result is not None:
                results.append(result)

    return pd.DataFrame(results) if results else pd.DataFrame()


# Main processing function without parallelism (sequential)
def main_sequential_processing(comid_stn, year, start_date, end_date):
    results = []
    
    for idx, row in comid_stn.iterrows():
        result = process_row(row, year, start_date, end_date, read_data, filter_and_save_data)
        if result is not None:
            results.append(result)

    return pd.DataFrame(results) if results else pd.DataFrame()


# Define the main function
def main(parallel=True):
    year = '1979'   # All data starting from 1979 till 2023 are stored in the folder 1979. This is for retrospective 3.0
    start_date = "1979-02-01"
    end_date = "2023-01-31"
    comid_stn = pd.read_excel('usgs_with_comid.xlsx')  # This is the file that contains the information of the COMIDs that we want to perform the analysis for 
    rpq_filtered = pd.read_csv('consolidated_annual_min_retro_3.csv')  # This file contains the annual maximum discharge for all the COMIDs sorted in ascending order
    comid_stn = pd.merge(comid_stn, rpq_filtered, on='COMID', how='inner')
    # comid_stn = comid_stn[0:10]  # Limiting for testing

    if parallel:
        # Run parallel processing
        final_results = main_parallel_processing(comid_stn, year, start_date, end_date)
    else:
        # Run sequential processing
        final_results = main_sequential_processing(comid_stn, year, start_date, end_date)

    # Save final results
    if not final_results.empty:
        output_folder_path = f'result_{year}'
        if not os.path.exists(output_folder_path):
            os.makedirs(output_folder_path)
        final_results.to_csv(f'error_{year}_nwm_3_0_with_metrics.csv')


# Profile the main function
if __name__ == "__main__":
    # Choose whether to run parallel or sequential processing
    cProfile.run('main(parallel=False)', 'performance_profile')  # If you have a large set of dataset, use parallel or sequential processing as per your choice
    # Analyze profiling stats
    p = pstats.Stats('performance_profile')
    p.strip_dirs().sort_stats('cumulative').print_stats(20)  
